In [91]:
import pandas as pd
import pvlib
from pathlib import Path
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

In [57]:
from pynasapower.get_data import query_power
from pynasapower.geometry import point
import pandas as pd, datetime
def send_req(points):
    df = query_power(
        geometry     = points,
        start        = datetime.date(2024,1,1),
        end          = datetime.date(2025,1,1),
        community    = "re",
        parameters   = params,
        temporal_api = "daily",
        spatial_api  = "point",
        format       = "csv",
        to_file      = False,
    )
    return df
    
params=["ALLSKY_SFC_SW_DWN","T2M","QV2M","PS","WS10M","CLRSKY_SFC_SW_DWN","ALLSKY_SFC_SW_DNI","ALLSKY_SFC_SW_DIFF"]
            #lon , lat
tbriz = point(44.24,39.26,"EPSG:4326")
kerman = point(57,30.2,"EPSG:4326")

In [ ]:
tbriz_df=send_req(tbriz)
kerman_df=send_req(kerman)

In [160]:
# 1. Load your dataset (skip NASA POWER header lines)
csv_in  = Path('./Dataset/kerman_w.csv')
csv_out = Path('./Dataset/kerman_power_predictions.csv')

df = pd.read_csv(csv_in)
df

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,T2M,QV2M,PS,WS10M,CLRSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF
0,2024,1,1,2.94,6.86,5.32,78.57,3.10,3.51,2.93,1.53
1,2024,1,2,3.36,6.41,4.25,78.55,1.50,3.75,4.44,1.35
2,2024,1,3,3.26,6.41,4.56,78.23,4.82,3.65,3.08,1.83
3,2024,1,4,2.84,5.01,4.25,78.27,3.12,3.76,2.80,1.66
4,2024,1,5,4.02,5.85,1.98,78.42,3.08,4.05,6.90,0.87
...,...,...,...,...,...,...,...,...,...,...,...
362,2024,12,28,3.66,6.21,1.85,78.73,4.65,3.77,5.02,1.27
363,2024,12,29,3.68,7.56,4.90,78.67,4.72,3.76,5.28,1.18
364,2024,12,30,2.42,6.49,5.74,78.59,5.02,3.63,0.47,1.96
365,2024,12,31,1.81,6.23,5.93,78.39,5.61,3.43,0.38,1.48


In [161]:
input_df=pd.DataFrame()
input_df["Date"] = pd.to_datetime({
    'year':  df['YEAR'],
    'month': df['MO'],
    'day':   df['DY']
})
input_df['ghi_Wm2'] = df['ALLSKY_SFC_SW_DWN'] * 1000.0 / 24.0
input_df['temp_air']   = df['T2M']
input_df['wind_speed'] = df['WS10M']
input_df["T2M"]=df["T2M"]

In [179]:
module_params = {
    'pdc0':      700, 
    'gamma_pdc': -0.0038,
}
inverter_params = {
    'pdc0': 5000,
    'eta_inv_nom': 0.96
}

In [180]:
dc_power = pvlib.pvsystem.pvwatts_dc(
    effective_irradiance=input_df['ghi_Wm2'],
    temp_cell=input_df["T2M"],     # uses temp_air & wind_speed internally if you pass them
    temp_ref=input_df['temp_air'],
    **module_params
)
ac_power = pvlib.inverter.pvwatts(
    pdc=dc_power,
    pdc0=inverter_params['pdc0'],
    eta_inv_nom=inverter_params['eta_inv_nom']
)

In [181]:
input_df['P (W)'] = ac_power
# input_df[["Date","P (W)"]].to_csv(csv_out,index=False)
input_df

,Date,ghi_Wm2,temp_air,wind_speed,T2M,P (W)
0,2024-01-01,122.500000,6.86,3.10,6.86,54.797328
1,2024-01-02,140.000000,6.41,1.50,6.41,66.819749
2,2024-01-03,135.833333,6.41,4.82,6.41,63.957355
3,2024-01-04,118.333333,5.01,3.12,5.01,51.934704
4,2024-01-05,167.500000,5.85,3.08,5.85,85.710167
...,...,...,...,...,...,...
362,2024-12-28,152.500000,6.21,4.65,6.21,75.406599
363,2024-12-29,153.333333,7.56,4.72,7.56,75.979038
364,2024-12-30,100.833333,6.49,5.02,6.49,39.911084
365,2024-12-31,75.416667,6.23,5.61,6.23,22.446483
